In [ ]:
!python -m pip install -q scramp==1.2.2 awswrangler==2.2.0

## Import required libraries including SageMaker Python SDK

In [ ]:
import pandas as pd
import numpy as np
import re
import json
import time
import uuid
from time import gmtime, strftime
import pathlib
from sklearn.model_selection import train_test_split

import boto3
import awswrangler as wr
import sagemaker
from sagemaker.workflow.pipeline import Pipeline
from sagemaker import get_execution_role
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker.xgboost.estimator import XGBoost
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.parameters import ParameterInteger, ParameterFloat, ParameterString
from sagemaker.workflow.steps import ProcessingStep, TrainingStep, CreateModelStep
from sagemaker.workflow.step_collections import RegisterModel

from sagemaker.sklearn.processing import SKLearnProcessor

from demo_helper import ModelMetrics

### Update SageMaker SDK if necessary 

In [ ]:
if int(sagemaker.__version__.split('.')[0]) != 2:
    !pip install sagemaker==2.24.1
    print("Updating SageMakerVersion. Please restart the kernel")
else:
    print("SageMaker SDK version is good")

### Set region, boto3 and SageMaker SDK variables

In [ ]:
boto_session = boto3.Session()
region = boto_session.region_name
print("Region = {}".format(region))

s3_client = boto3.client('s3', region_name=region)

sagemaker_boto_client = boto_session.client('sagemaker')

sagemaker_session = sagemaker.session.Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_boto_client)

sagemaker_role = sagemaker.get_execution_role()

### Create directories in the SageMaker default bucket for this tutorial¶


In [ ]:
default_bucket= sess.default_bucket() # Alterantively you can use our custom bucket here. 

prefix = 'sagemaker-tutorial' # use this prefix to store all files pertaining to this workshop.

dataprefix = prefix + '/data'
traindataprefix = prefix + '/train_data'
testdataprefix = prefix + '/test_data'
testdatanolabelprefix = prefix + '/test_data_no_label'
trainheaderprefix = prefix + '/train_headers'


training_job_output_path = f's3://{default_bucket}/{prefix}/training_jobs'

create_dataset_script_uri = f's3://{default_bucket}/{prefix}/code/create_dataset.py'
deploy_model_script_uri = f's3://{default_bucket}/{prefix}/code/deploy_model.py'

processing_dir = "/opt/ml/processing"


#======> variables used for parameterizing the notebook run
flow_instance_count = 1
flow_instance_type = "ml.m5.4xlarge"


deploy_model_instance_type = "ml.m4.xlarge"


# Preprocess data

Use the following code snippet to download the dataset to /data/ folder

In [ ]:
!mkdir ./data
!wget -O ./data/default_of_credit_card.xls  https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls

In [ ]:
# change column names and save the file as .csv
data_path = './data/default_of_credit_card.xls'

df = pd.read_excel('./data/default_of_credit_card.xls', header=1)

df.head()

In [ ]:
timestamp = pd.to_datetime('now').timestamp()
df['EVENT_TIME'] = timestamp

df = df.astype(np.float64)

For Amazon SageMaker built-in XGBoost algorithm, the label column needs to be the first column in the dataframe. Use the code below to make that change:

In [ ]:
cols = list(df)
cols.insert(0, cols.pop(cols.index('default payment next month')))
df = df.loc[:, cols]

In [ ]:
df.rename(columns={"default payment next month": "LABEL"}, inplace=True)

Now we need to upload the data to S3. Also after running the cell below, you can continue the guide using the provided `./data/dataset.csv`

In [ ]:
df.to_csv('./data/dataset.csv', index=False)

response = sess.upload_data('./data/dataset.csv', bucket=default_bucket, key_prefix=dataprefix)
print(response)

### create train and test data set

In [ ]:
df_train = df.sample(frac=.80, random_state=42)
df_test = df.drop(df_train.index)

# save train and test data locally
df_train.to_csv('./data/train.csv', index=False)
df_test.to_csv('./data/test.csv', index=False)

# upload train and test data to s3
response = sagemaker_session.upload_data('./data/train.csv', bucket=default_bucket, key_prefix=dataprefix)
train_data_uri = response

response = sagemaker_session.upload_data('./data/test.csv', bucket=default_bucket, key_prefix=dataprefix)
test_data_uri = response

# SageMaker Feature Store

Amazon SageMaker Feature Store is a purpose-built repository where you can store and access features so it’s much easier to name, organize, and reuse them across teams. SageMaker Feature Store provides a unified store for features during training and real-time inference without the need to write additional code or create manual processes to keep features consistent. SageMaker Feature Store keeps track of the metadata of stored features (e.g. feature name or version number) so that you can query the features for the right attributes in batches or in real time using Amazon Athena, an interactive query service. SageMaker Feature Store also keeps features updated, because as new data is generated during inference, the single repository is updated so new features are always available for models to use during training and inference.

A feature store consists of an offline componet stored in S3 and an online component stored in a low-latency database. The online database is optional, but very useful if you need supplemental features to be available at inference. In this section, we will create a feature groups for our Claims and Customers datasets. After inserting the claims and customer data into their respective feature groups, you need to query the offline store with Athena to build the training dataset.

You can reference the [SageMaker Developer Guide](https://docs.aws.amazon.com/sagemaker/latest/dg/feature-store.html) for more information about the SageMaker Feature Store.

In [ ]:
featurestore_runtime = boto_session.client(
    service_name='sagemaker-featurestore-runtime', 
    region_name=region
)

feature_store_session = sagemaker.Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_boto_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime
)

## Configure the feature groups

When you set up your feature groups, you need to customize the feature names with a unique name and set up each feature group with the FeatureGroup class. 


The datatype for each feature is set by passing a dataframe and inferring the proper datatype. Feature data types can also be set via a config variable, but it will have to match the correspongin Python data type in the Pandas dataframe when it's ingested to the Feature Group.

In [ ]:
fg_name = f'credit-default' + strftime('%d-%H-%M-%S', gmtime())

credit_feature_group = FeatureGroup(
    name=fg_name, 
    sagemaker_session=feature_store_session)

# You can now load the feature definitions by passing a data frame containing the feature data.
credit_feature_group.load_feature_definitions(data_frame=df);

# Create the feature groups

You must tell the Feature Group which columns in the dataframe correspond to the required record indentifier and event time features.

In [ ]:
print(f"{fg_name} is the feature group name in use")

In this step, you use the create function to create the feature group. The following code shows all of the available parameters. The online store is not created by default, so you must set this as True if you want to enable it. The s3_uri is the S3 bucket location of your offline store.

In [ ]:
# record identifier and event time feature names
record_identifier_feature_name = 'ID'
event_time_feature_name = 'EVENT_TIME'


# check if the feature groups is created successfully 
def wait_for_feature_group_creation_complete(feature_group):
    status = feature_group.describe().get("FeatureGroupStatus")
    while status == "Creating":
        print("Waiting for Feature Group Creation")
        time.sleep(5)
        status = feature_group.describe().get("FeatureGroupStatus")
    if status != "Created":
        raise RuntimeError(f"Failed to create feature group {feature_group.name}")
    print(f"FeatureGroup {feature_group.name} successfully created.")
    

print(f"\n Using s3://{default_bucket}/{prefix}")
credit_feature_group.create(
    s3_uri=f"s3://{default_bucket}/{prefix}",
    record_identifier_name=record_identifier_feature_name,
    event_time_feature_name=event_time_feature_name,
    role_arn=sagemaker_role,
    enable_online_store=True
)

wait_for_feature_group_creation_complete(feature_group=credit_feature_group)

In [ ]:
credit_feature_group.describe()

### Ingest records into the Feature Groups

After the Feature Groups have been created, we can put data into each store by using the PutRecord API. This API can handle high TPS and is designed to be called by different streams. The data from all of these Put requests is buffered and written to s3 in chunks. The files will be written to the offline store within a few minutes of ingestion.

In [ ]:
if 'credit_table' in locals():
    print("You may have already ingested the data into your Feature Groups. If you'd like to do this again, you can run the ingest methods outside of the 'if/else' statement.")

else:
    credit_feature_group.ingest(
    data_frame=df, max_workers=5, wait=True
    );

# Create a SageMaker Pipeline to Automate All the Steps from Data Prep to Model Deployment
Now that youve manually done each step in our machine learning workflow, you can create a pipeline which trains a new model, persists the model in SageMaker and then adds the model to the registry.

### Pipeline parameters
An important feature of SageMaker Pipelines is the ability to define the steps ahead of time, but be able to change the parameters to those steps at execution without having to re-define the pipeline. This can be achieved by using ParameterInteger, ParameterFloat or ParameterString to define a value upfront which can be modified when you call pipeline.start(parameters=parameters) later. Only certain parameters can be defined this way.

In [ ]:
train_instance_param = ParameterString(
    name="TrainingInstance",
    default_value="ml.m4.xlarge"
)

model_approval_status = ParameterString(
    name="ModelApprovalStatus",
    default_value="PendingManualApproval"
)

dw_flow = './credit_default_flow/credit_default.flow'
# # set an output path where the trained model will be saved
# model_prefix = 'xgboost'
# output_path = 's3://{}/{}/output'.format(default_bucket, prefix)

# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
xgboost_container = sagemaker.image_uris.retrieve("xgboost", region, "1.2-1")

# Step 1: Train XGBoost Model

In [ ]:
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":"50"}

train_instance_count = 1
train_instance_type = "ml.m4.xlarge"
content_type = "text/csv"


# construct a SageMaker estimator that calls the xgboost-container
xgb_estimator = sagemaker.estimator.Estimator(image_uri=xgboost_container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          instance_count=train_instance_count, 
                                          instance_type=train_instance_type, 
                                          volume_size=5, # 5 GB 
                                          output_path=training_job_output_path)

train_step = TrainingStep(
    name='XgboostTrain',
    estimator=xgb_estimator,
    inputs={
        'train': TrainingInput(train_data_uri, content_type=content_type)
    }
)

# Step 2: Model Pre-Deployment Step

In [ ]:
model = sagemaker.model.Model(
    name='credit-default-demo-pipeline-xgboost',
    image_uri=train_step.properties.AlgorithmSpecification.TrainingImage,
    model_data=train_step.properties.ModelArtifacts.S3ModelArtifacts,
    sagemaker_session=sagemaker_session,
    role=sagemaker_role
)

inputs = sagemaker.inputs.CreateModelInput(
    instance_type="ml.m4.xlarge"
)

create_model_step = CreateModelStep(
    name="ModelPreDeployment",
    model=model,
    inputs=inputs
)

# Step 3: Run Bias Metrics with Clarify

In [ ]:
# clarify config
bias_report_output_path = f's3://{default_bucket}/{prefix}/clarify-output/bias'
s3_client = boto3.client('s3', region_name=region)


bias_data_config = sagemaker.clarify.DataConfig(
    s3_data_input_path=train_data_uri,
    label='LABEL',
    dataset_type='text/csv',
s3_output_path=bias_report_output_path)

bias_config = sagemaker.clarify.BiasConfig(
    label_values_or_threshold=[0],
    facet_name='SEX',
    facet_values_or_threshold=[1])

analysis_config = bias_data_config.get_config()
analysis_config.update(bias_config.get_config())
analysis_config["methods"] = {"pre_training_bias": {"methods": "all"}}

clarify_config_dir = pathlib.Path('config')
clarify_config_dir.mkdir(exist_ok=True)
with open(clarify_config_dir / 'analysis_config.json', 'w') as f:
    json.dump(analysis_config, f)
    
s3_client.upload_file(Filename='config/analysis_config.json', Bucket=default_bucket, Key=f'{prefix}/clarify-config/analysis_config.json')

In [ ]:
# clarify processing step

clarify_processor = sagemaker.processing.Processor(
    base_job_name='fraud-detection-demo-clarify-processor',
    image_uri=sagemaker.clarify.image_uris.retrieve(framework='clarify', region=region),
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type='ml.c5.xlarge')

clarify_step = ProcessingStep(
    name="ClarifyProcessor",
    processor=clarify_processor,
    inputs=[
        sagemaker.processing.ProcessingInput(
            input_name="analysis_config",
            source=f's3://{default_bucket}/{prefix}/clarify-config/analysis_config.json',
            destination="/opt/ml/processing/input/config"),
        sagemaker.processing.ProcessingInput(
            input_name="dataset",
            source=train_data_uri,
            destination="/opt/ml/processing/input/data")  
    ],
    outputs=[
        sagemaker.processing.ProcessingOutput(
            source="/opt/ml/processing/output/analysis.json",
            destination=bias_report_output_path,
            output_name="analysis_result")
    ]
)

# Step 4: Register Model

In [ ]:
model_metrics = ModelMetrics(
    bias=sagemaker.model_metrics.MetricsSource(
        s3_uri=clarify_step.properties.ProcessingOutputConfig.Outputs['analysis_result'].S3Output.S3Uri,
        content_type="application/json"
    )
)

if 'mpg_name' not in locals():
    mpg_name = prefix
    %store mpg_name
    print(f'Model Package Group name: {mpg_name}')
    
register_step = RegisterModel(
    name="XgboostRegisterModel",
    estimator=xgb_estimator,
    model_data=train_step.properties.ModelArtifacts.S3ModelArtifacts,
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.t2.medium", "ml.m5.xlarge"],
    transform_instances=["ml.m5.xlarge"],
    model_package_group_name=mpg_name,
    approval_status=model_approval_status,
    model_metrics=model_metrics
)

# Step 5: Deploy Model

In [ ]:
endpoint_name = "xgboost-model-pipeline-0120"

s3_client.upload_file(Filename='deploy_model.py', Bucket=default_bucket, Key=f'{prefix}/code/deploy_model.py')

deploy_model_processor = SKLearnProcessor(
    framework_version='0.23-1',
    role=sagemaker_role,
    instance_type="ml.t3.medium",
    instance_count=1,
    base_job_name='fraud-detection-demo-deploy-model',
    sagemaker_session=sagemaker_session)

deploy_step = ProcessingStep(
    name='DeployModel',
    processor=deploy_model_processor,
    job_arguments=[
        "--model-name", create_model_step.properties.ModelName, 
        "--region", region,
        "--endpoint-instance-type", deploy_model_instance_type,
        "--endpoint-name", endpoint_name],
    code=deploy_model_script_uri)

# Combine the Pipeline Steps and Run

In [ ]:
pipeline_name = f'credit-default'
%store pipeline_name

pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        train_instance_param, 
        model_approval_status],
    steps=[
        train_step, 
        create_model_step, 
        clarify_step, 
        register_step,
        deploy_step
    ])

## Submit the pipeline definition to the SageMaker Pipeline service¶

Note: If an existing pipeline has the same name it will be overwritten.


In [ ]:
pipeline.upsert(role_arn=sagemaker_role)

In [ ]:
start_response = pipeline.start()

start_response.wait()
start_response.describe()

#  View results of Clarify job

# Make a prediction

In [ ]:
predictor = sagemaker.predictor.Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker_session)

#### Sample an application from the test data and get it's features from online feature store

In [ ]:
sample_id = df_test.sample(1).iloc[0]['ID']

response = featurestore_runtime.get_record(
        FeatureGroupName=fg_name, 
        RecordIdentifierValueAsString= str(sample_id)

    )

In [ ]:
df_sample = pd.DataFrame(response['Record']).set_index('FeatureName').drop('LABEL')
data_input = ','.join(df_sample['ValueAsString'])

In [ ]:
results = predictor.predict(data_input, initial_args = {"ContentType": "text/csv"})
prediction = json.loads(results)

print (f'Probablitity of the default payment next month for ID:{int(sample_id)} is : {round(prediction * 100)}%')

# Clean up

After running the demo, you should remove the resources which were created. You can also delete all the objects in the project's S3 directory by passing the keyword argument delete_s3_objects=True.



In [ ]:
from demo_helper import delete_project_resources

In [ ]:
# delete_project_resources(
#     sagemaker_boto_client=sagemaker_boto_client,
#     endpoint_name=endpoint_name, 
#     pipeline_name=pipeline_name, 
#     mpg_name=mpg_name, 
#     prefix=prefix,
#     delete_s3_objects=False,
#     bucket_name=default_bucket)